In [11]:
import pandas as pd

df = pd.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

In [12]:
colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

This is the part where we analyze the data.

In [17]:
import duckdb

def dependency(country, year, item, threshold):
    query = '''
    select Partner_Countries,
        sum(Value) as trade_quantity,
        round(trade_quantity /
        (
            select
                sum(Value)
            from df
            where Year = {year}
            and Reporter_Countries = '{country}'
            and Item = '{item}'
            and Element = 'Import Quantity'
        ), 2) as dep_ratio
    from df
    where Year = {year}
    and Reporter_Countries = '{country}'
    and Element = 'Import Quantity'
    and Item = '{item}'
    and Value > 0
    group by Partner_Countries
    having dep_ratio >= {threshold}
    order by dep_ratio desc; 
'''.format(item = item, country = country, year = year, threshold = threshold)
    
    return duckdb.sql(query).fetchdf()

dependency('United Arab Emirates', 2021, 'Wheat', 0.05)

,Partner_Countries,trade_quantity,dep_ratio
0,India,436264.45,0.31
1,Russian Federation,288637.00,0.20
2,Australia,241280.00,0.17
3,Canada,209260.10,0.15
4,Romania,156790.00,0.11


In [33]:
exp = duckdb.query('''
select Partner_Countries,
            sum(Value) as trade
        from df
        where Year = 2019
        and Reporter_Countries = 'India'
        and Element = 'Export Quantity'
        and Item = 'Rice'
        and Value > 0
        group by Partner_Countries
        order by trade desc;
''').fetchdf()

imp = duckdb.query('''
    select Reporter_Countries,
        sum(Value) as imports
    from df
    where Year = 2019
    and Element = 'Import Quantity'
    and Item = 'Rice'
    and Value > 0
    group by Reporter_Countries
    order by imports desc;
''').fetchdf()

joined = duckdb.query('''
    select * from exp
    left join imp
    on exp.Partner_Countries = imp.Reporter_Countries
    order by trade desc;
''').fetchdf()

joined

,Partner_Countries,trade,Reporter_Countries,imports
0,Nepal,234406.94,Nepal,265486.18
1,Philippines,7201.54,Philippines,27643.60
2,Bangladesh,4430.02,Bangladesh,5101.79
3,Viet Nam,395.31,Viet Nam,16379.25
4,United Arab Emirates,184.54,United Arab Emirates,1010.61
5,Bahrain,110.07,Bahrain,143.98
6,Australia,96.32,Australia,166.24
7,Pakistan,87.73,Pakistan,10475.00
8,Bhutan,76.00,Bhutan,76.00
9,United States of America,54.46,United States of America,177.02


In [22]:
# def leverage():
#     query = '''
#         select Partner_Countries,
#             sum(Value) as trade
#         from df
#         where Year = 2019
#         and Reporter_Countries = 'India'
#         and Element = 'Export Quantity'
#         and Item = 'Rice'
#         and Value > 0
#         group by Partner_Countries
#         order by trade desc;
#     '''
#     return duckdb.sql(query)

# leverage()

,Partner_Countries,trade
0,Nepal,234406.94
1,Philippines,7201.54
2,Bangladesh,4430.02
3,Viet Nam,395.31
4,United Arab Emirates,184.54
5,Bahrain,110.07
6,Australia,96.32
7,Pakistan,87.73
8,Bhutan,76.00
9,United States of America,54.46
